# Memoria TFM Eduardo Hernanz - David Sánchez

### El presente documento trata de guiar paso a paso al lector para poder comprender las decisiones tomadas tanto a nivel de 'negocio' como técnico
### El proyecto consiste en el desarrollo de un simulador que permita optimizar el tiempo empleado por un triatleta para recorrer el segmento de ciclismo de un triatlon
### Como requisito es necesario contar con un potenciómetro instalado en la bicicleta que medirá el grado de esfuerzo realizado
### Asímismo, el simulador adaptará la estrategia de potencias según los tramos caracterizados por su pendiente y viento aparente, dando una distribución de potencias que cumpla con una media previamente definida por el usuario y no sobrepasando en ningún momento una potencia máxima

#### Para conseguir este resultado, se han desarrollado las siguientes piezas de código
#### 0 - Requisitos: 
Entornos:

    Python 3.6

    Anaconda 5.2

Lista de paquetes y orden necesarios (pip install):

    msgpack

    geopy

    ipynb

    python-metar

    tensorflow

    keras
    
    plotly

Otro Software

    Tableau 10.5 o superior
   
Conexión a internet para ejecutar la parte de Análisis Track puesto que se conecta para recuperar la información climática
#### 1 - Scraping de las condiciones climatológicas de los entrenamientos usados para entrenar el modeloVer el notebook ../Entrenamientos/Analisis Track (Machine Learning) N_files.ipynb
Ver el Notebook: ../Entrenamientos/Weather_History.ipynb
#### 2 - Limpieza y creación de 'features' en los entrenamientos
Ver el Notebook: ../Entrenamientos/Analisis Track (Machine Learning) N_files.ipynb
#### 3 - Red Neuronal para estimar la velocidad media necesaria para recorrer un tramo en función de la potencia aplicada, el desnivel y el viento aparente
Ver más abajo
#### 4 - Simulador que prueba sobre cada tramo del recorrido objetivo diferentes potencias con el fin de minimizar el tiempo total cumpliendo con una potencia media objetivo
Ver más abajo

#### La Potencia viene dada por la siguiente formula

$P = \frac{\rho}{2}(V+wV)^2VCdA+(Cr+\%)mgV$

Dónde:

P es la potencia necesaria

$\rho$ es la densidad del aire

V es la velocidad

wV es velocidad el viento

Cd es el coeficiente de drag

A es el área efectiva

Cr es el coeficiente de rozamiento

$\%$ es la pendiente

m es la masa total del conjunto bicicleta - ciclista

#### Para comprender la importancia de los factores que intervienen podemos dividirla en la potencia necesaria para vencer la resistencia aerodinámica y la potencia para vencer factores como el rozamiento y la gravedad

La potencia total es:

$P = \frac{\rho}{2}(V+Wv)^2VCdA+(Cr+\%)mgV$

Y se puede descomponer en la potencia necesaria para vencer al viento y para vencer el rozamiento

Potencia Aerodinámica: $Pa = \frac{\rho}{2}(V+Wv)^2VCdA$

Potencia Rozamiento: $Pr = (Cr)mgV$

Potencia Gravedad: $Pg = (\%)mgV$

In [3]:
import plotly as py
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import HTML
import math as mt
py.offline.init_notebook_mode(connected=True)

import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
import math as mt

In [15]:
def show_graph(CdA,Pendiente,Viento):

    V = np.arange(0,60,0.5)

    ro = 1.2
    #cda = 0.3
    cr = 0.0033
    m = 90
    g = 9.81
    s = mt.sin(mt.atan(Pendiente/100))

    Pa = ro/2*CdA*((V/3.6)+(Viento/3.6))**2*(V/3.6)
    Pr = (cr)*m*g*(V/3.6)
    Pg = (s)*m*g*(V/3.6)
    Pt = Pa + Pr + Pg

    trace0 = go.Scatter(
        x=V,
        y=Pa,
        mode='lines',
        name='Pot_aerodinamica'
    )

    trace1 = go.Scatter(
        x=V,
        y=Pr,
        mode='lines',
        name='Pot_rozamiento'
    )

    trace2 = go.Scatter(
        x=V,
        y=Pg,
        mode='lines',
        name='Pot_pendinte'
    )
    
    trace3 = go.Scatter(
        
        x=V,
        y=Pt,
        mode='lines',
        name='Pot_total'
    )
    
    layout = go.Layout(title='Simulador de potencia (Potencia vs. Velocidad)')
    
    data = [trace0, trace1, trace2, trace3]

    fig = dict(data=data, layout=layout)

    py.offline.iplot(fig)
widgets.interact(show_graph, CdA=(0.1,0.5,0.05), Pendiente=(-10,10,0.5), Viento=(-15,15,1))

interactive(children=(FloatSlider(value=0.30000000000000004, description='CdA', max=0.5, min=0.1, step=0.05), …

<function __main__.show_graph(CdA, Pendiente, Viento)>

#### Como puede observarse en el gráfico anterior, la potencia necesaria para incrementar la velocidad depende sobre todo de la velocidad a la que estemos circulando
#### Si además hay aire en contra (Viento>0), el problema se agrava
#### En cambio, con el % de pendiente, se ve que los watios 'invertidos' se transforman casi linealmente en velocidad, puesto que la parte aerodinámica tiene menos influencia al circular a baja velocidad
#### Por tanto, parece que existe una oportunidad clara de repartir los watios del ciclista en los tramos más favorables, ahorrando en aquellos en los que los vamos a perder mayoritariamente luchando contra el viento o la pendiente
#### Si ahora pintamos sólamente la potencia aerodinámica y vemos como varía en función de la velocidad y el viento, obtenemos lo siguiente

In [5]:
def show_graph():

    V = np.arange(0,40,0.5)
    wind = np.arange(0,30,0.5)

    ro = 1.2
    cda = 0.25
    
    z = []

    for v in V:
        new_row = []
        for w in wind:
            new_row.append( ro/2*cda*((v/3.6)+(w/3.6))**2*(v/3.6) )
        z.append(list(new_row))

    trace0 = go.Heatmap(
        x=V,
        y=wind,
        z=z,
        colorscale='Jet'
    )

    data = [trace0]
    
    layout = go.Layout(title='Análisis de Potencia Aerodinámica (Viento(y) vs Velocidad(x))')

    fig = dict(data=data, layout=layout)

    py.offline.iplot(fig)

widgets.interact(show_graph)

<function __main__.show_graph>

#### Podemos apreciar como con aire en contra, debemos reducir mucho la velocidad para mantener la potencia constante

## Una vez que hemos visto los factores que intervienen en la problemática, tenemos que tratar de obtener los valores de CdA y Cr

#### En el notebook "RandomForestRegressor (CdA y Cr, P en funcion V)" se trata de obtener valores de CdA y Cr. Sin embargo, los resultados no son concluyentes, debido a que las mediciones de CdA y Cr deben realizarse en condiciones controladas
Ver el Notebook: ../Machine Learning/RandomForestRegressor (CdA y Cr, P en funcion V).ipynb
#### Tras los intentos de averiguar el CdA y el Cr y dado que el objetivo final es tener una alta capacidad de predicción, hemos cambiado el enfoque. Por este motivo se han probado redes neuronales que han demostrado tener un poder predictivo muy alto
Ver el Notebook: ../RNN/TF.ipynb

## En este punto, ya tenemos los datos analizados, limpios, enriquecidos y hemos desarrollado una Red Neuronal que nos da unos buenos valores predictivos
## Podemos por tanto, plantearnos desarrollar un simulador que asigne potencias a cada tramo de forma que se minimice el tiempo total cumpliendo ciertas restricciones de potencia
## Sin embargo antes de entrar en este punto conviene explicar el método seguido para inicializar las potencias de simulación. No es lo mismo una etapa con un puerto de 1h de duración y luego una bajada, que una en la que ese mismo desnivel se recorra en sucesivos toboganes por ser una carretera ondulada


In [4]:
def graph(pot_min, pot_max, ff_1, ff_2):
    porc = np.arange(-10,10,0.5)

    potencia = pot_min+(pot_max-pot_min)/(1+ff_1*np.e**(ff_2*porc*-1))
    potencia_min = pot_min+(pot_max-25-pot_min)/(1+ff_1*np.e**(ff_2*porc*-1))
    potencia_max = pot_min+25+(pot_max-pot_min-25)/(1+ff_1*np.e**(ff_2*porc*-1))

    trace0 = go.Scatter(        
            x=porc,
            y=potencia,
            mode='lines',
            name='Potencia_obj')

    trace1 = go.Scatter(        
            x=porc,
            y=potencia_min,
            mode='lines',
            name='Potencia_min')

    trace2 = go.Scatter(        
            x=porc,
            y=potencia_max,
            mode='lines',
            name='Potencia_max')




    layout = go.Layout(title='Sigmoide de potencia')

    data = [trace0, trace1, trace2]

    fig = dict(data=data, layout=layout)

    py.offline.iplot(fig)
    
widgets.interact(graph, pot_min=(40,100,5), pot_max=(250,300,5), ff_1=(0.1,1,0.1), ff_2=(0.1,1,0.1))

<function __main__.graph>

### Supongamos un ciclista con una potencia máxima para la simulación de 275w y una potencia mínima de 70w. Podemos jugar con los factores de forma de la sigmoide para variar la forma en la que inicializamos la potencia
### El factor_forma_1 (ff_1) nos sirve para desplazar hacia las pendientes positivas el momento en que aumenta la potencia del ciclista. Usaremos valores grandes cuando por ejemplo, las bajadas del recorrido sean muy curveadas en las que no tiene sentido tratar de entregar potencia cuesta abajo
### El factor_forma_2 (ff_2) nos sirve aumentar la pendiente con la que pasamos de valores de potencia bajos a altos. Usaremos valores elevados en perfiles muy llanos, puesto que la mayor parte del tiempo la pasaremos en torno al 0%. En etapas de montaña, sería un 'suicidio' deportivo usar un ff_2 alto, pues nos haría pasar muchas horas en valores máximos de potencia que no es posible aguantar
### En general estos 4 valores, junto con el resultado de la simulación que se explica más adelante, deben ser usados de forma iterativa hasta obtener unos valores que aporten confianza al entrenador y al triatleta
Ver el Notebook: ../Machine Learning/Warm-Start.ipynb


## Ya tenemos todos los componentes para pasar a la simulación
Ver el Notebook: ../Simulador/Simulacion.ipynb